# Most Active Users

## 1. Initial Setup

**Assumption:**

1. I will define the most active users as the one with the highest number of post, comments and likes.
2. The rank of importance will given by addin a higher weight to each activity count as it follows:

    - Post:     3 points
    - Comment:  2 points
    - Like:     1 point


In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Needed Packages
import pandas as pd
import os
import numpy as np
from datetime import datetime

# Visualization packages
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

# Display settings
import warnings
warnings.filterwarnings('ignore')
#%matplotlib widget

### 1.1 Import User, Post and Comment Data

In [18]:
# Import users data
path = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\production_datasets\\cleaned_datasets"
file_name = "users_cleaned.csv"
file_path = os.path.join(path, file_name)
df_users = pd.read_csv(file_path)

# Keep just rows with active in the 'state' column
df_users = df_users[df_users['state'] == 'active']
df_users.head()

,first_login,suspended,username,id,last_skipped_contact_details,state,role,last_login,updated,used,display_name_extension,suspended_at,language,created,department,location,position
0,2022-12-16 19:26:22,False,Aaron_Burch,093c7a08-e32c-40eb-9eb6-a5e26c623be0,2024-02-05 21:31:03,active,member,2024-02-05 21:31:00,2024-02-05 21:31:00,2024-03-05 22:26:00,"Advanced Crew Leader, MD",NaN,en,2022-12-16 18:02:51,Operations,MD,Advanced Crew Leader
1,2024-02-20 21:19:59,False,Aaron_Cassatt,c9c1ce3b-2afa-453d-9c72-b521442b288f,2024-02-20 21:20:18,active,member,2024-02-20 21:19:59,2024-02-21 03:01:52,2024-03-05 19:28:00,"Crew Member, NEPA",NaN,en,2024-02-20 20:01:20,Operations,NEPA,Crew Member
2,2023-12-11 20:19:18,False,Aaron_Ervin,eae780a3-9c68-457b-ad3f-c38210a46539,2023-12-11 20:20:07,active,member,2023-12-11 20:19:18,2024-02-26 22:01:04,2024-03-05 22:24:00,"Crew Leader, WNC",NaN,en,2023-12-11 18:02:40,Operations,WNC,Crew Leader
3,2023-07-24 19:06:39,False,Aaron_Hunter,a7bb6f33-fec7-41a7-9ebf-22479dbcb747,2023-07-24 19:07:07,active,member,2023-07-24 19:06:39,2024-02-27 11:35:40,2024-03-05 00:41:00,"Crew Leader, WNC",NaN,en,2023-07-24 16:04:41,Operations,WNC,Crew Leader
4,2024-02-13 19:24:10,False,Aaron_Lewis,7058dc0e-5e81-4ccc-8975-9df990372494,2024-02-13 19:24:33,active,member,2024-02-13 19:24:10,2024-02-14 23:10:56,2024-03-05 22:12:00,"Crew Member, ATL",NaN,en,2024-02-13 18:02:14,Operations,ATL,Crew Member


In [24]:
# Import post
file_name = "posts_cleaned.csv"
file_path = os.path.join(path, file_name)
df_posts = pd.read_csv(file_path)
df_posts.head() 

,stream_name,stream_id,post_id,action,created,title,text,like_count,comment_countt,mentions,labels,username,author_user_id,author_position,author_status
0,Jobs,9254,7615837,POSTED,2024-03-05 14:57:18,Cm Available anytime!,WNC CM available!,0,0,NaN,NaN,Lisa_Muegge,d52d678b-df08-4b69-9454-06ce860e0206,Crew Member,active
1,"OSC, We Can Help",22708,7615091,POSTED,2024-03-05 12:39:48,Report,I'm reporting a serious matter I'm done with t...,2,2,NaN,NaN,Linsey_McGregor,8db5cc91-01fa-40bb-bbd0-218846cf229a,Advanced Crew Leader,active
2,Jobs,9254,7615025,POSTED,2024-03-05 12:32:24,OPEN JOBS - NOVA - 2 CM,1 CM\nClient: Washington Gas\nDate: 3/5/24\nAd...,3,1,NaN,NaN,client_services_am,af09ca5c-59cd-427c-96be-9bc49f613b28,NaN,active
3,Water Break,6787,7610693,POSTED,2024-03-05 12:25:23,Week 9 of Our 2024 In-Field Promotions | Congr...,"Congratulations, CLs, ACLs, and other new prom...",6,4,"Rizear_Law,Joe_Hall,Leon_Witherspoon,Virginia_...",NaN,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
4,Jobs,9254,7614975,POSTED,2024-03-05 12:08:57,NaN,Available for emergencies…NOVA 🚨🏮🚥💦☔️❄️🤤👷🏾,4,0,NaN,NaN,Justin_House,5c1223af-c738-498b-84d6-455a9080c914,Crew Leader,active


In [6]:
# Import comments
file_name = "comments_cleaned.csv"
file_path = os.path.join(path, file_name)
df_comments = pd.read_csv(file_path)
df_comments.head() 

,stream_name,created_date,post_id,comment_id,comment_text,like_count,report_count,username,author_user_id,author_position,author_status
0,Water Break,2024-03-05 15:15:55,7612625,6866587,Congratulations and a great addition to the te...,0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
1,Water Break,2024-03-05 15:10:47,7612663,6866567,"Hello team, remember to have safety discussion...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
2,Water Break,2024-03-05 15:08:29,7613861,6866561,"@Larry_Broderick, both our moderation team and...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
3,Safety & Operations,2024-03-05 15:04:42,7613237,6866545,"@William_Pulliam, we appreciate you being prou...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
4,Safety & Operations,2024-03-05 15:02:47,7613255,6866539,"We appreciate you supporting @Dennis_Cumbie, @...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active


### 1.2 Import and Process  Comment's Likes Data

In [ ]:
# Import comment's likes data
path = "C:\\Users\\eduar\\Documents\\Master_Thesis\\GenAI_Thesis_Beekeeper\\data\\production_datasets\\FlaggerForce\\StreamContent"
file_name = "historical_comments_likes_2022_2024feb.csv"
file_path = os.path.join(path, file_name)
df_comment_likes = pd.read_csv(file_path)

# Delete duplicates
df_comment_likes.drop_duplicates(keep="first")
# Change date format to datetime
df_comment_likes['created_at'] = pd.to_datetime(df_comment_likes['created_at'], format='ISO8601')

# date range of df_comment_likes
print("The date range of comment's likes is:")
print(f"Min date: {df_comment_likes['created_at'].min()}")
print(f"Max date: {df_comment_likes['created_at'].max()}")
print(f"Date range: {df_comment_likes['created_at'].max() - df_comment_likes['created_at'].min()}")

print(f"Shape of the dataset: {df_comment_likes.shape}")
df_comment_likes.head()

The date range of comment's likes is:
Min date: 2022-01-01 00:04:07
Max date: 2024-03-05 15:06:59
Date range: 794 days 15:02:52
Shape of the dataset: (88612, 7)


,id,commentid,userid,created_at,username,display_name,Author Position
0,5582555,6866506,b701ab9f-563a-4425-a389-aff803a8da58,2024-03-05 15:06:59,FlaggerForce,NaN,NaN
1,5582553,6866380,b701ab9f-563a-4425-a389-aff803a8da58,2024-03-05 15:06:58,FlaggerForce,NaN,NaN
2,5582536,6865995,b701ab9f-563a-4425-a389-aff803a8da58,2024-03-05 15:01:40,FlaggerForce,NaN,NaN
3,5582535,6866414,b701ab9f-563a-4425-a389-aff803a8da58,2024-03-05 15:01:10,FlaggerForce,NaN,NaN
4,5582449,6866139,b701ab9f-563a-4425-a389-aff803a8da58,2024-03-05 14:22:26,FlaggerForce,NaN,NaN


### 1.3 Import and Process Post's Likes Data

In [84]:
# Import post's likes data
file_name = "historical_Post_likes_2022_2024Feb.csv"
file_path = os.path.join(path, file_name)
df_post_likes = pd.read_csv(file_path)
print(f"Shape of the dataset: {df_post_likes.shape}")
df_post_likes.head()

Shape of the dataset: (1048575, 7)


,stream_name,stream_id,post_id,username,display_name,action,created
0,Jobs,9254,7614948,Lisa_Muegge,Lisa Muegge,liked post,2024-03-05T15:19:33
1,"OSC, We Can Help",22708,7614948,Lisa_Muegge,Lisa Muegge,liked post,2024-03-05T15:19:33
2,Flagger Force Connect,5476,7614948,Lisa_Muegge,Lisa Muegge,liked post,2024-03-05T15:19:33
3,Safety & Operations,5745,7614948,Lisa_Muegge,Lisa Muegge,liked post,2024-03-05T15:19:33
4,Safety & Operations,5745,7614948,Lisa_Muegge,Lisa Muegge,liked post,2024-03-05T15:19:33


In [89]:
# Change date format to datetime
df_post_likes['created'] = pd.to_datetime(df_post_likes['created'])

# date range of df_post_likes
print("The date range of posts is:")
print(f"Min date: {df_post_likes['created'].min()}")
print(f"Max date: {df_post_likes['created'].max()}")
print(f"Date range: {df_post_likes['created'].max() - df_post_likes['created'].min()}")

The date range of posts is:
Min date: 2024-03-05 14:18:28
Max date: 2024-03-05 15:19:33
Date range: 0 days 01:01:05


In [55]:
df_post_likes.sample(50)

,stream_name,stream_id,post_id,username,display_name,action,created
384221,Water Break,6787,7613237,FlaggerForce,NaN,liked post,2024-03-05T15:04:47
482950,Safety & Operations,5745,7614975,Ebony_Scott,Ebony Scott,liked post,2024-03-05T14:35:05
727849,Water Break,6787,7615091,Harold_Friscia,Harold Friscia,liked post,2024-03-05T14:18:50
45097,Jobs,9254,7614948,Lisa_Muegge,Lisa Muegge,liked post,2024-03-05T15:19:33
278643,Water Break,6787,7612663,FlaggerForce,NaN,liked post,2024-03-05T15:09:17
794021,Jobs,9254,7615091,Harold_Friscia,Harold Friscia,liked post,2024-03-05T14:18:50
308396,Jobs,9254,7612663,FlaggerForce,NaN,liked post,2024-03-05T15:09:17
752295,Water Break,6787,7615091,Harold_Friscia,Harold Friscia,liked post,2024-03-05T14:18:50
153780,Water Break,6787,7612625,FlaggerForce,NaN,liked post,2024-03-05T15:10:55
644934,Jobs,9254,7610693,FlaggerForce,NaN,liked post,2024-03-05T14:21:49


In [86]:
len(df_post_likes['post_id'].unique())

8

In [73]:
df = df_post_likes.copy()
# Delete columns stream_name and stream_id
#df.drop(columns=['stream_name', 'stream_id'], inplace=True)
#df.reset_index(drop=True, inplace=True)
# Delete duplicates
df2 = df.drop_duplicates(subset=["post_id","username"], keep="first")

print(f"Shape of the dataset: {df2.shape}")
df2

Shape of the dataset: (9, 7)


,stream_name,stream_id,post_id,username,display_name,action,created
0,Jobs,9254,7614948,Lisa_Muegge,Lisa Muegge,liked post,2024-03-05T15:19:33
119592,Jobs,9254,7612625,FlaggerForce,NaN,liked post,2024-03-05T15:10:55
239184,Jobs,9254,7612663,FlaggerForce,NaN,liked post,2024-03-05T15:09:17
358776,Jobs,9254,7613237,FlaggerForce,NaN,liked post,2024-03-05T15:04:47
478368,Jobs,9254,7614975,Ebony_Scott,Ebony Scott,liked post,2024-03-05T14:35:05
597960,Jobs,9254,7610693,FlaggerForce,NaN,liked post,2024-03-05T14:21:49
717552,Jobs,9254,7615091,Harold_Friscia,Harold Friscia,liked post,2024-03-05T14:18:50
837144,Jobs,9254,7615025,Harold_Friscia,Harold Friscia,liked post,2024-03-05T14:18:40
956736,Jobs,9254,7614975,Harold_Friscia,Harold Friscia,liked post,2024-03-05T14:18:28


In [63]:
# Delete columns stream_name and stream_id
df_post_likes.drop(columns=['stream_name', 'stream_id'], inplace=True)
df_post_likes.reset_index(drop=True, inplace=True)
# Delete duplicates
df_post_likes.drop_duplicates(inplace=True)

df_post_likes.head(50)


,post_id,username,display_name,action,created
0,7614948,Lisa_Muegge,Lisa Muegge,liked post,2024-03-05 15:19:33
119592,7612625,FlaggerForce,NaN,liked post,2024-03-05 15:10:55
239184,7612663,FlaggerForce,NaN,liked post,2024-03-05 15:09:17
358776,7613237,FlaggerForce,NaN,liked post,2024-03-05 15:04:47
478368,7614975,Ebony_Scott,Ebony Scott,liked post,2024-03-05 14:35:05
597960,7610693,FlaggerForce,NaN,liked post,2024-03-05 14:21:49
717552,7615091,Harold_Friscia,Harold Friscia,liked post,2024-03-05 14:18:50
837144,7615025,Harold_Friscia,Harold Friscia,liked post,2024-03-05 14:18:40
956736,7614975,Harold_Friscia,Harold Friscia,liked post,2024-03-05 14:18:28


## 2. Find Most Active Users


In [34]:
# Posts by user
posts_by_user = df_posts['username'].value_counts().to_frame().reset_index()
# Comments by user
comments_by_user = df_comments['username'].value_counts().to_frame().reset_index()
# Post's likes by user
posts_likes_by_user = df_post_likes['username'].value_counts().to_frame().reset_index()
# Comment's likes by user
comments_likes_by_user = df_comment_likes['username'].value_counts().to_frame().reset_index()


# Merge users with count of posts
df_active_users = df_users.merge(posts_by_user[['username', 'count']], left_on='username', right_on='username', how='left')
df_active_users.rename(columns={'count':'posts'}, inplace=True) 
df_active_users['posts'] = df_active_users['posts'].fillna(0)
df_active_users['posts'] = df_active_users['posts'].astype(int)

# Merge users with count of comments
df_active_users = df_active_users.merge(comments_by_user[['username', 'count']], left_on='username', right_on='username', how='left')
df_active_users.rename(columns={'count':'comments'}, inplace=True) 
df_active_users['comments'] = df_active_users['comments'].fillna(0)
df_active_users['comments'] = df_active_users['comments'].astype(int)

# Merge users with count of post's likes
df_active_users = df_active_users.merge(posts_likes_by_user[['username', 'count']], left_on='username', right_on='username', how='left')
df_active_users.rename(columns={'count':'post_likes'}, inplace=True)
df_active_users['post_likes'] = df_active_users['post_likes'].fillna(0)
df_active_users['post_likes'] = df_active_users['post_likes'].astype(int)

# Merge users with count of comment's likes
df_active_users = df_active_users.merge(comments_likes_by_user[['username', 'count']], left_on='username', right_on='username', how='left')
df_active_users.rename(columns={'count':'comment_likes'}, inplace=True)
df_active_users['comment_likes'] = df_active_users['comment_likes'].fillna(0)
df_active_users['comment_likes'] = df_active_users['comment_likes'].astype(int)

df_active_users.head()

,first_login,suspended,username,id,last_skipped_contact_details,state,role,last_login,updated,used,display_name_extension,suspended_at,language,created,department,location,position,posts,comments,post_likes,comment_likes
0,2022-12-16 19:26:22,False,Aaron_Burch,093c7a08-e32c-40eb-9eb6-a5e26c623be0,2024-02-05 21:31:03,active,member,2024-02-05 21:31:00,2024-02-05 21:31:00,2024-03-05 22:26:00,"Advanced Crew Leader, MD",NaN,en,2022-12-16 18:02:51,Operations,MD,Advanced Crew Leader,14,117,0,17
1,2024-02-20 21:19:59,False,Aaron_Cassatt,c9c1ce3b-2afa-453d-9c72-b521442b288f,2024-02-20 21:20:18,active,member,2024-02-20 21:19:59,2024-02-21 03:01:52,2024-03-05 19:28:00,"Crew Member, NEPA",NaN,en,2024-02-20 20:01:20,Operations,NEPA,Crew Member,0,1,0,0
2,2023-12-11 20:19:18,False,Aaron_Ervin,eae780a3-9c68-457b-ad3f-c38210a46539,2023-12-11 20:20:07,active,member,2023-12-11 20:19:18,2024-02-26 22:01:04,2024-03-05 22:24:00,"Crew Leader, WNC",NaN,en,2023-12-11 18:02:40,Operations,WNC,Crew Leader,0,2,0,0
3,2023-07-24 19:06:39,False,Aaron_Hunter,a7bb6f33-fec7-41a7-9ebf-22479dbcb747,2023-07-24 19:07:07,active,member,2023-07-24 19:06:39,2024-02-27 11:35:40,2024-03-05 00:41:00,"Crew Leader, WNC",NaN,en,2023-07-24 16:04:41,Operations,WNC,Crew Leader,0,0,0,1
4,2024-02-13 19:24:10,False,Aaron_Lewis,7058dc0e-5e81-4ccc-8975-9df990372494,2024-02-13 19:24:33,active,member,2024-02-13 19:24:10,2024-02-14 23:10:56,2024-03-05 22:12:00,"Crew Member, ATL",NaN,en,2024-02-13 18:02:14,Operations,ATL,Crew Member,2,16,0,39


In [36]:
df_active_users['role'].unique()

array(['member', 'admin'], dtype=object)

In [37]:
# posts_by_user[posts_by_user['username']=='Aaron_Burch']

new_columns = ['first_login', 'username', 'id', 'posts', 'comments', 
               'post_likes', 'comment_likes', 'department', 'location', 'position']
df_active_users = df_active_users[new_columns]
df_active_users

,first_login,username,id,posts,comments,post_likes,comment_likes,department,location,position
0,2022-12-16 19:26:22,Aaron_Burch,093c7a08-e32c-40eb-9eb6-a5e26c623be0,14,117,0,17,Operations,MD,Advanced Crew Leader
1,2024-02-20 21:19:59,Aaron_Cassatt,c9c1ce3b-2afa-453d-9c72-b521442b288f,0,1,0,0,Operations,NEPA,Crew Member
2,2023-12-11 20:19:18,Aaron_Ervin,eae780a3-9c68-457b-ad3f-c38210a46539,0,2,0,0,Operations,WNC,Crew Leader
3,2023-07-24 19:06:39,Aaron_Hunter,a7bb6f33-fec7-41a7-9ebf-22479dbcb747,0,0,0,1,Operations,WNC,Crew Leader
4,2024-02-13 19:24:10,Aaron_Lewis,7058dc0e-5e81-4ccc-8975-9df990372494,2,16,0,39,Operations,ATL,Crew Member
...,...,...,...,...,...,...,...,...,...,...
2090,2023-04-17 19:57:40,Zachery_Siers,480432ff-60b5-4cf8-ba26-21d888971d6f,63,51,0,35,Operations,CPA,Advanced Crew Leader
2091,2024-02-02 20:39:33,Zamari_Morrison,e4ef985b-3b6f-42b1-b365-a25aa63b35c1,0,0,0,0,Operations,SFL,Crew Member
2092,2022-05-06 19:36:15,Zeke_Harris,e607c588-e715-42d0-8954-2b6a81d7dd60,8,15,0,2,Operations,MD,Advanced Crew Leader
2093,2021-08-20 20:22:29,Zoe_Hughes,21a2335d-4d4b-4ef4-b79b-527c2be13695,62,141,0,304,Operations,WPA,Advanced Crew Leader


In [39]:
# Find the most active users
df_active_users['total_activity'] = df_active_users['posts']*3 + df_active_users['comments']*2 + df_active_users['post_likes'] + df_active_users['comment_likes']
df_active_users.sort_values('total_activity', ascending=False).head(20)

,first_login,username,id,posts,comments,post_likes,comment_likes,department,location,position,total_activity
703,2017-12-20 04:34:31,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,1005,3270,478368,5271,NaN,NaN,NaN,493194
757,2022-04-05 19:51:17,Harold_Friscia,f75b4e38-55ef-4cd1-99e4-2c969f54acc9,1,501,331023,44,Operations,SEPA,Crew Member,332072
627,2020-05-22 20:05:56,Ebony_Scott,8a900aa7-4e78-41e8-ba57-e8c8c5a4936b,207,544,119592,418,Operations,NOVA,Crew Member,121719
1316,2024-02-29 16:24:47,Lisa_Muegge,d52d678b-df08-4b69-9454-06ce860e0206,3,4,119592,1,Operations,WNC,Crew Member,119610
2037,2019-12-23 15:47:56,weekend_dispatch,cd381448-d11f-41b6-b15c-b2dd0a884bf5,2802,3194,0,3280,Operations,OSC,Weekend Dispatch,18074
391,2020-11-09 22:27:49,client_services_transition,f206abd1-e46c-4d35-be75-a3b937edf885,3600,1272,0,594,NaN,NaN,NaN,13938
389,2020-11-10 10:49:16,client_services_am,af09ca5c-59cd-427c-96be-9bc49f613b28,2208,191,0,216,NaN,NaN,NaN,7222
1130,2022-08-30 18:58:13,Karen_Stroup,e31dc6f4-a5b3-46e2-994c-7b8ad8f82910,176,2487,0,96,Operations,CPA,Crew Leader,5598
404,2021-12-14 20:37:50,Connard_Edlin,7ad38938-6791-4099-bf0b-c972af801104,332,1563,0,871,Operations,WPA,Crew Member,4993
390,2020-11-09 22:33:52,client_services_pm,e7a068df-4acf-4164-b303-0cbce29e3c6b,884,366,0,777,NaN,NaN,NaN,4161
